In [1]:
import zipfile
import os
import numpy as np
from keras.preprocessing.image import load_img, img_to_array
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score


In [2]:
# Unzip the archive
local_zip = 'D:/Code/python/mat3533/practice08/data/cats_and_dogs_filtered.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall()
zip_ref.close()

In [3]:
base_dir = 'D:/Code/python/mat3533/practice08/exercise/ex01/cats_and_dogs_filtered'
# Change the base_dir to where you put dataset
print("Contents of base directory:")
print(os.listdir(base_dir))
print("\nContents of train directory:")
print(os.listdir(f'{base_dir}\\train'))
print("\nContents of validation directory:")
print(os.listdir(f'{base_dir}\\validation'))

Contents of base directory:
['train', 'validation', 'vectorize.py']

Contents of train directory:
['cats', 'dogs']

Contents of validation directory:
['cats', 'dogs']


In [4]:
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')
# Directory with training cat/dog pictures
train_cats_dir = os.path.join(train_dir, 'cats')
train_dogs_dir = os.path.join(train_dir, 'dogs')
# Directory with validation cat/dog pictures
validation_cats_dir = os.path.join(validation_dir, 'cats')
validation_dogs_dir = os.path.join(validation_dir, 'dogs')
print("\nContents of train directory:")
print(os.listdir(f'{base_dir}\\train'))
print("\nContents of validation directory:")
print(os.listdir(f'{base_dir}\\validation'))


Contents of train directory:
['cats', 'dogs']

Contents of validation directory:
['cats', 'dogs']


In [5]:
train_cat_fnames = os.listdir( train_cats_dir )
train_dog_fnames = os.listdir( train_dogs_dir )
print(train_cat_fnames[:10])
print(train_dog_fnames[:10])
print('total training cat images :', len(os.listdir( train_cats_dir ) ))
print('total training dog images :', len(os.listdir( train_dogs_dir ) ))
print('total validation cat images :', len(os.listdir( validation_cats_dir ) ))
print('total validation dog images :', len(os.listdir( validation_dogs_dir ) ))

['cat.0.jpg', 'cat.1.jpg', 'cat.10.jpg', 'cat.100.jpg', 'cat.101.jpg', 'cat.102.jpg', 'cat.103.jpg', 'cat.104.jpg', 'cat.105.jpg', 'cat.106.jpg']
['dog.0.jpg', 'dog.1.jpg', 'dog.10.jpg', 'dog.100.jpg', 'dog.101.jpg', 'dog.102.jpg', 'dog.103.jpg', 'dog.104.jpg', 'dog.105.jpg', 'dog.106.jpg']
total training cat images : 1000
total training dog images : 1000
total validation cat images : 500
total validation dog images : 500


Đưa ảnh về kích thước 150px x 150px

In [6]:
def resize_images(directory):
    for filename in os.listdir(directory):
        path = os.path.join(directory, filename)
        img = load_img(path, target_size=(150, 150))
        img_array = img_to_array(img)
        img_array /= 255.0  # Normalize pixel values
        np.save(f"{path[:-4]}.npy", img_array)  # Save the image array as numpy array
        os.remove(path)  # Remove the original image

resize_images(train_cats_dir)
resize_images(train_dogs_dir)
resize_images(validation_cats_dir)
resize_images(validation_dogs_dir)

Gán nhãn 0 cho cat, 1 cho dog

In [7]:
train_images = []
train_labels = []
validation_images = []
validation_labels = []

for filename in os.listdir(train_cats_dir):
    train_images.append(np.load(os.path.join(train_cats_dir, filename)))
    train_labels.append(0)  # 0 for cats

for filename in os.listdir(train_dogs_dir):
    train_images.append(np.load(os.path.join(train_dogs_dir, filename)))
    train_labels.append(1)  # 1 for dogs

for filename in os.listdir(validation_cats_dir):
    validation_images.append(np.load(os.path.join(validation_cats_dir, filename)))
    validation_labels.append(0)

for filename in os.listdir(validation_dogs_dir):
    validation_images.append(np.load(os.path.join(validation_dogs_dir, filename)))
    validation_labels.append(1)

Chuyển ảnh thành vector

In [8]:
train_images = np.array(train_images)
train_labels = np.array(train_labels)
validation_images = np.array(validation_images)
validation_labels = np.array(validation_labels)

train_images_flatten = train_images.reshape(train_images.shape[0], -1)
validation_images_flatten = validation_images.reshape(validation_images.shape[0], -1)

Chạy mô hình Logistic Regression

In [9]:
model = LogisticRegression()
model.fit(train_images_flatten, train_labels)

validation_predictions = model.predict(validation_images_flatten)

# Evaluate accuracy
accuracy = accuracy_score(validation_labels, validation_predictions)
precision = precision_score(validation_labels, validation_predictions)
recall = recall_score(validation_labels, validation_predictions)
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)

c:\Users\Admin\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy: 0.551
Precision: 0.5578231292517006
Recall: 0.492


Chuẩn bị dữ liệu test mới


In [13]:
from PIL import Image

# Đường dẫn tới thư mục chứa ảnh
directory = 'D:/Code/python/mat3533/practice08/data/test_dog_cat'

# Tạo một danh sách để lưu trữ các tên file ảnh
image_files = []

# Lặp qua các số từ 1 đến 20 để tìm tên ảnh
for i in range(1, 21):
    # Tạo tên file ảnh dựa trên số thứ tự
    filename = f'pic{i}.jpg'
    # Kiểm tra xem file có tồn tại trong thư mục không
    if os.path.exists(os.path.join(directory, filename)):
        # Nếu tồn tại, thêm vào danh sách
        image_files.append(filename)

# Khởi tạo danh sách để lưu trữ dữ liệu hình ảnh
test_new_images = []

# Lặp qua các tên file ảnh đã tìm được
for filename in image_files:
    # Đọc ảnh từ file
    img = Image.open(os.path.join(directory, filename))
    # Resize ảnh về kích thước cố định, ví dụ (100, 100)
    img = img.resize((150, 150))
    # Chuyển ảnh thành mảng numpy
    img_array = np.array(img)
    # Thêm mảng numpy này vào danh sách dữ liệu
    test_new_images.append(img_array)

# Chuyển danh sách dữ liệu thành mảng numpy
test_new_images = np.array(test_new_images)

# Giảm số chiều về 225 chiều
test_new_images = test_new_images.reshape(20, -1)

# print(test_new_images.shape)
test_new_images.shape

(20, 67500)

Chạy dự đoán dữ liệu test mới

In [14]:
y_pred = model.predict(test_new_images)
y_pred

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0])